<a href="https://colab.research.google.com/github/stellaevat/ontology-mapping/blob/main/colabs/load_and_extract.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
!pip install pronto owlready2 \
&& wget -O doid.obo https://gla-my.sharepoint.com/:u:/g/personal/2526934t_student_gla_ac_uk/EfUC_RdrfZdOsOrtmNATjuoBPDaIkSTUMyxJXyO2KKC6yw?download=1 \
&& wget -O ncit.obo https://gla-my.sharepoint.com/:u:/g/personal/2526934t_student_gla_ac_uk/ETmaJIC0fAlItdsp8WQxS_wBzKN_6x08EZrtsOxVnbzvSg?download=1 \
&& wget -O doid.owl https://gla-my.sharepoint.com/:u:/g/personal/2526934t_student_gla_ac_uk/EVjp-NoZlZtDq42LqxZH08ABz2h7IWqdw2gxetzvA-u3hQ?download=1

--2023-11-06 19:31:33--  https://gla-my.sharepoint.com/:u:/g/personal/2526934t_student_gla_ac_uk/EfUC_RdrfZdOsOrtmNATjuoBPDaIkSTUMyxJXyO2KKC6yw?download=1
Resolving gla-my.sharepoint.com (gla-my.sharepoint.com)... 13.107.136.10, 13.107.138.10, 2620:1ec:8f8::10, ...
Connecting to gla-my.sharepoint.com (gla-my.sharepoint.com)|13.107.136.10|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /personal/2526934t_student_gla_ac_uk/Documents/UG%20COMPSCI/Year%204/Individual%20Project/ontologies/doid.obo?ga=1 [following]
--2023-11-06 19:31:35--  https://gla-my.sharepoint.com/personal/2526934t_student_gla_ac_uk/Documents/UG%20COMPSCI/Year%204/Individual%20Project/ontologies/doid.obo?ga=1
Reusing existing connection to gla-my.sharepoint.com:443.
HTTP request sent, awaiting response... 200 OK
Length: 6634417 (6.3M) [application/octet-stream]
Saving to: ‘doid.obo’

doid.obo            100%[===================>]   6.33M  2.93MB/s    in 2.2s    

2023-11-06 19:31:37 (2.93 

In [54]:
import pronto
import owlready2
import numpy as np
from tqdm import tqdm
from collections import Counter

In [33]:
ncit = pronto.Ontology("ncit.obo")
doid = pronto.Ontology("doid.obo")
doid_owl = owlready2.get_ontology("doid.owl").load()

# Extract equivalences

In [100]:
# Get Property object with the given name, from the given owlready2 Ontology

def get_owl_property(onto, target_prop_name):
  all_props = list(onto.properties())
  target_prop = None
  for prop in all_props:
    if prop.python_name == target_prop_name:
      target_prop = prop
      break
  return target_prop

print("Found: ", get_owl_property(doid_owl, "hasDbXref"))

Found:  oboInOwl.hasDbXref


In [101]:
# Correct the ontology code in the given entity id

def correct_code(entity_id, corrections={"NCI" : "NCIT"}, delimiter=":"):
  code, num = entity_id.split(delimiter)
  if code in corrections:
    return corrections[code] + delimiter + num
  else:
    return entity_id

# Get entity equivalences encoded by the given Property name to the given target ontology, in the given owlready2 Ontology

def get_owl_entity_equivalences(source_onto, target_onto_code="NCI:", xref_prop_name="hasDbXref"):
  all_equivalences = {}
  xref_prop = get_owl_property(source_onto, xref_prop_name)

  if xref_prop:
    for entity in list(source_onto.classes()):
      if xref_prop in entity.get_properties(entity):
        xrefs = [correct_code(ref) for ref in xref_prop[entity] if target_onto_code in ref]
        # Only keep equivalences with single target
        if xrefs and len(xrefs) == 1:
          all_equivalences[correct_code(entity.id[0])] = xrefs[0]
  return all_equivalences

In [102]:
def write_mappings(mappings, filepath):
  with open(filepath, "w") as f:
    for source_id, target_id in mappings.items():
      f.write(f"{source_id},{target_id}\n")

In [103]:
all_equivalences = get_owl_entity_equivalences(doid_owl)
# print("Equivalences (DOID to NCIT): ", all_equivalences)
print("Total equivalences: ", len(all_equivalences))

Total equivalences:  4331


In [104]:
write_mappings(all_equivalences, "equiv_doid2ncit.csv")
write_mappings({v:k for (k,v) in all_equivalences.items()}, "equiv_ncit2doid.csv")

# Derive subsumptions

In [112]:
def get_subsumptions_from_equivalences(equivalences, source_onto, target_onto, negative=False, reverse=False):
  np.random.seed(3)
  source_ids = [term.id for term in source_onto.terms()]
  target_ids = [term.id for term in target_onto.terms()]
  subsumptions = {}

  for source_id, target_id in tqdm(equivalences.items()):
    if source_id in source_onto.terms():
      source_entity = source_onto.get_term(source_id)
      source_parents = source_entity.superclasses(distance=1, with_self=False)

      if target_id in target_onto.terms():
        target_entity = target_onto.get_term(target_id)
        target_parents = target_entity.superclasses(distance=1, with_self=False)

        # Only keep mappings with single parents
        if target_parents and source_parents and len(set(target_parents)) == 1 and len(set(source_parents)) == 1:

          if not negative:
            if reverse:
              subsumptions[target_id] = set(source_parents).pop().id
            else:
              subsumptions[source_id] = set(target_parents).pop().id

          else:
            if reverse:
              source_parent_id = list(source_parents)[0].id
              neg_sub = source_ids[np.random.randint(0, len(source_ids))]
              while neg_sub == source_parent_id:
                neg_sub = source_ids[np.random.randint(0, len(source_ids))]
              subsumptions[target_id] = neg_sub
            else:
              target_parent_id = list(target_parents)[0].id
              neg_sub = target_ids[np.random.randint(0, len(target_ids))]
              while neg_sub == target_parent_id:
                neg_sub = target_ids[np.random.randint(0, len(target_ids))]
              subsumptions[source_id] = neg_sub
  print()
  return subsumptions

In [113]:
doid_to_ncit_subs = get_subsumptions_from_equivalences(all_equivalences, doid, ncit)
# print("Subsumption relations (DOID to NCIT): ", doid_to_ncit_subs)
print("Total subsumptions (DOID to NCIT): ", len(doid_to_ncit_subs))

100%|██████████| 4331/4331 [00:00<00:00, 8483.95it/s]



Total subsumptions (DOID to NCIT):  1883


In [114]:
neg_doid_to_ncit_subs = get_subsumptions_from_equivalences(all_equivalences, doid, ncit, negative=True)
# print("Negative subsumption relations (DOID to NCIT): ", neg_doid_to_ncit_subs)
print("Total negative subsumptions (DOID to NCIT): ", len(neg_doid_to_ncit_subs))

100%|██████████| 4331/4331 [00:00<00:00, 7329.69it/s]


Total negative subsumptions (DOID to NCIT):  1883


In [115]:
ncit_to_doid_subs = get_subsumptions_from_equivalences(all_equivalences, doid, ncit, reverse=True)
# print("Subsumption relations (NCIT to DOID): ", ncit_to_doid_subs)
print("Total subsumptions (NCIT to DOID): ", len(ncit_to_doid_subs))

100%|██████████| 4331/4331 [00:00<00:00, 4988.32it/s]


Total subsumptions (NCIT to DOID):  1873


In [116]:
neg_ncit_to_doid_subs = get_subsumptions_from_equivalences(all_equivalences, doid, ncit, negative=True, reverse=True)
# print("Negative subsumption relations (NCIT to DOID): ", neg_ncit_to_doid_subs)
print("Total negative subsumptions (NCIT to DOID): ", len(neg_ncit_to_doid_subs))

100%|██████████| 4331/4331 [00:00<00:00, 5270.07it/s]



Total negative subsumptions (NCIT to DOID):  1873


In [117]:
write_mappings(doid_to_ncit_subs, "subs_doid2ncit.csv")
write_mappings(neg_doid_to_ncit_subs, "neg_subs_doid2ncit.csv")
write_mappings(ncit_to_doid_subs, "subs_ncit2doid.csv")
write_mappings(neg_ncit_to_doid_subs, "neg_subs_ncit2doid.csv")